In [1]:
import pandas as pd
import numpy as np
import io
import requests

In [11]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_7294a4b37bcb4e21b8dcf59d30d33325 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='LDHXc9O7JsAGbrUlhd04wGcgu4826HlV73TaKmUqegpy',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_7294a4b37bcb4e21b8dcf59d30d33325.get_object(Bucket='neighborhood-donotdelete-pr-1cvufevkdk0ksi',Key='ALL.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

D = df_data_1

In [12]:
#D = pd.read_csv("ALL.csv")
D = D[['Postcode', 'Borough' ,'Neighbourhood' ,'Latitude' ,'Longitude']]

In [13]:
D.Borough.unique()

array(['Etobicoke', 'East York', 'Scarborough', 'North York',
       'Downtown Toronto', 'York', 'Central Toronto', 'East Toronto',
       'West Toronto'], dtype=object)

In [14]:
TORRONTO = ['East Toronto','West Toronto','Downtown Toronto']
D_TO = D[D.Borough.isin(['East Toronto','West Toronto','Downtown Toronto'])]
D_NY = D.loc[D['Borough'] == 'North York']

In [18]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests # library to handle requests
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize

print('Folium installed')
print('Libraries imported.')
print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [19]:
latitude = 43.71269
longitude = -79.509461
NY_map = folium.Map(location=[latitude, longitude], zoom_start=11)
NY_map

In [20]:
Neighborhood = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(D_NY.Latitude, D_NY.Longitude):
    Neighborhood.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

for lat, lng, in zip(D_TO.Latitude, D_TO.Longitude):
    Neighborhood.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='Red',
            fill=True,
            fill_color='Red',
            fill_opacity=0.6
        )
    )    

# add incidents to map
NY_map.add_child(Neighborhood)

In this map, red dots are in **Torronto** and Blue dots are in **North York**

Setting up API for data acquisition from the **Foursquare**

In [21]:
CLIENT_ID = '12LBRK2WPAN0OBGDRD2T5XUWIRTHO44T0RUMVQEJKAM0ODPE' # your Foursquare ID
CLIENT_SECRET = 'BJTOBFGIDZXB5I0HHJR54C5ZSWH3NONJWJ3PGXX4OQLM4EVP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
radius = 3000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 12LBRK2WPAN0OBGDRD2T5XUWIRTHO44T0RUMVQEJKAM0ODPE
CLIENT_SECRET:BJTOBFGIDZXB5I0HHJR54C5ZSWH3NONJWJ3PGXX4OQLM4EVP


In [22]:
df1=pd.DataFrame()
for latitude, longitude, in zip(D_NY.Latitude, D_NY.Longitude):
    search_query = 'Bakery'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    df1= df1.append(dataframe)
    
df2=pd.DataFrame()
for latitude, longitude, in zip(D_TO.Latitude, D_TO.Longitude):
    search_query = 'Bakery'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    df2= df2.append(dataframe)


Classfting Dataframe into **Torronto** and **North York**

In [23]:
dataframe_NY = df1
dataframe_TO = df2

In [24]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_NY.columns if col.startswith('location.')] + ['id']
dataframe_filtered_NY = dataframe_NY.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_NY['categories'] = dataframe_filtered_NY.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_NY.columns = [column.split('.')[-1] for column in dataframe_filtered_NY.columns]


In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_TO.columns if col.startswith('location.')] + ['id']
dataframe_filtered_TO = dataframe_TO.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_TO['categories'] = dataframe_filtered_TO.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_TO.columns = [column.split('.')[-1] for column in dataframe_filtered_TO.columns]


## Heat Map for distribution **Bakeries** between Neibourhoods

In [26]:
VENS = dataframe_filtered_TO.append(dataframe_filtered_NY)
from folium import plugins
from folium.plugins import HeatMap

latitude = 43.71269
longitude = -79.509461
NY_map = folium.Map(location=[latitude, longitude], zoom_start=11)


Neighborhood = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(D_NY.Latitude, D_NY.Longitude):
    Neighborhood.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='blue',
            fill=False,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

for lat, lng, in zip(D_TO.Latitude, D_TO.Longitude):
    Neighborhood.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='Red',
            fill=False,
            fill_color='Pink',
            fill_opacity=0.8
        )
    )    

for lat, lng, in zip(VENS.lat, VENS.lng):
    Neighborhood.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=2, # define how big you want the circle markers to be
            color='gray',
            fill=False,
            fill_color='gray',
            fill_opacity=0.6
        )
    )      
stationArr = VENS[['lat', 'lng']].values.tolist()
NY_map.add_child(Neighborhood)
HeatMap(stationArr).add_to(NY_map)
#stationArr = VENS[['lat', 'lng']].values.tolist()
#Neighborhood.add_child(plugins.HeatMap(stationArr, radius=15))

NY_map

In this Fig, grey dots are the bakeires, Red and Blue Circles are the Neighborhood in the Torronto and North York, Respectively.
It can be seen that the "bakeries" in the **Torronto** are more concentrated while it is more scatered in the **North York**

It is also notocible that the Bakeries are greater in number in **Torronto** than the **North York**

Some neighborhood are more close to **bakeries** than other neighborhoods

## Anlysis for the Proximity to the Grocery Stores

In [28]:

df1=pd.DataFrame()
for latitude, longitude, in zip(D_NY.Latitude, D_NY.Longitude):
    search_query = 'grocery'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    df1= df1.append(dataframe)
    
df2=pd.DataFrame()
for latitude, longitude, in zip(D_TO.Latitude, D_TO.Longitude):
    search_query = 'grocery'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    df2= df2.append(dataframe)


In [29]:
dataframe_NY = df1
dataframe_TO = df2

In [30]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_NY.columns if col.startswith('location.')] + ['id']
dataframe_filtered_NY = dataframe_NY.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_NY['categories'] = dataframe_filtered_NY.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_NY.columns = [column.split('.')[-1] for column in dataframe_filtered_NY.columns]


In [31]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_TO.columns if col.startswith('location.')] + ['id']
dataframe_filtered_TO = dataframe_TO.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_TO['categories'] = dataframe_filtered_TO.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_TO.columns = [column.split('.')[-1] for column in dataframe_filtered_TO.columns]


In [18]:
VENS = dataframe_filtered_TO.append(dataframe_filtered_NY)
VENS.reset_index()

from folium import plugins
from folium.plugins import HeatMap

latitude = 43.71269
longitude = -79.509461
NY_map = folium.Map(location=[latitude, longitude], zoom_start=11)


Neighborhood = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(D_NY.Latitude, D_NY.Longitude):
    Neighborhood.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='blue',
            fill=False,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

for lat, lng, in zip(D_TO.Latitude, D_TO.Longitude):
    Neighborhood.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='Red',
            fill=False,
            fill_color='Pink',
            fill_opacity=0.8
        )
    )    

for lat, lng, in zip(VENS.lat, VENS.lng):
    Neighborhood.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=2, # define how big you want the circle markers to be
            color='gray',
            fill=False,
            fill_color='gray',
            fill_opacity=0.6
        )
    )      
stationArr = VENS[['lat', 'lng']].values.tolist()
NY_map.add_child(Neighborhood)
HeatMap(stationArr).add_to(NY_map)
#stationArr = VENS[['lat', 'lng']].values.tolist()
#Neighborhood.add_child(plugins.HeatMap(stationArr, radius=15))


In [32]:
NY_map

This heatmap shows the **neighborhood** and the datapoints are the avaialble **grocery stores** within 2000 meter of each neigbourhood
The results indicate that the Genrally neighborhoods in Torronto Borrogh are more close to the grocery stores than the other ones in North York

To focus more on the closeness of the neighborhood to the grocery stores, k-means clustering algorthim was to categorize differnce site based on the closeness to the grocery stors

In [33]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

Libraries imported.


**10** Clusters was initially assumed

In [34]:
k_means = KMeans(init="k-means++", n_clusters=10, n_init=12)
k_means.fit(stationArr)
k_means_labels = k_means.labels_
LABELS = pd.DataFrame({'LAB': k_means_labels})
LABELS = LABELS.applymap(str)


In [35]:
from folium import plugins
from folium.plugins import HeatMap

latitude = 43.71269
longitude = -79.509461
NY_map = folium.Map(location=[latitude, longitude], zoom_start=11)


Neighborhood = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(D_NY.Latitude, D_NY.Longitude):
    Neighborhood.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='blue',
            fill=False,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

for lat, lng, in zip(D_TO.Latitude, D_TO.Longitude):
    Neighborhood.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='Red',
            fill=False,
            fill_color='Pink',
            fill_opacity=0.8
        )
    ) 


    
NY_map.add_child(Neighborhood)

Gropus = plugins.MarkerCluster().add_to(NY_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(VENS.lat, VENS.lng, LABELS.LAB):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(Gropus)
    
    


NY_map


This map shows the concentration of the grocery stores on the map based on the Assigned category